# Convolutional Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

import os
from os import listdir
print(os.listdir("data"))

['test', 'test1.zip', 'train', 'train.zip']


In [2]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)
training_set = train_datagen.flow_from_directory('data/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary',
                                                 subset='training')
validation_set = train_datagen.flow_from_directory('data/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary',
                                                 subset='validation')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('data/test',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 0 images belonging to 0 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation = 'relu', input_shape= [64,64,3]))

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [43]:
cnn.fit(x = training_set, validation_data = validation_set, epochs = 10)

Epoch 1/10
625/625 [==============================] - 122s 195ms/step - loss: 0.3338 - accuracy: 0.8534 - val_loss: 0.4104 - val_accuracy: 0.8214
Epoch 2/10
625/625 [==============================] - 114s 183ms/step - loss: 0.3134 - accuracy: 0.8642 - val_loss: 0.3885 - val_accuracy: 0.8292
Epoch 3/10
625/625 [==============================] - 111s 178ms/step - loss: 0.3025 - accuracy: 0.8679 - val_loss: 0.4018 - val_accuracy: 0.8170
Epoch 4/10
625/625 [==============================] - 113s 180ms/step - loss: 0.2951 - accuracy: 0.8711 - val_loss: 0.3974 - val_accuracy: 0.8324
Epoch 5/10
625/625 [==============================] - 113s 180ms/step - loss: 0.2811 - accuracy: 0.8838 - val_loss: 0.4118 - val_accuracy: 0.8288
Epoch 6/10
625/625 [==============================] - 114s 182ms/step - loss: 0.2654 - accuracy: 0.8886 - val_loss: 0.4015 - val_accuracy: 0.8256
Epoch 7/10
625/625 [==============================] - 120s 192ms/step - loss: 0.2546 - accuracy: 0.8902 - val_loss: 0.4022 -

## Part 4 - Making a single prediction

In [44]:
import numpy as np
from keras.preprocessing import image

ans = []
for i in range(1,12501):
    test_image = image.load_img(f'data/test/{i}.jpg', target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    training_set.class_indices
    ans.append(result[0][0])

In [45]:
sub = np.around(ans)
print(sub)

[1. 1. 1. ... 1. 1. 0.]


In [46]:
sub = pd.DataFrame(sub)
#ans = ans.assign(Survived = y_pred)
sub.to_csv('Submission.csv')
sub

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,0.0
...,...
12495,0.0
12496,0.0
12497,1.0
12498,1.0


In [47]:
cnn.save('convNN')

INFO:tensorflow:Assets written to: convNN\assets
